<a href="https://colab.research.google.com/github/pedroeli07/house_prices/blob/main/GradientBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
# Importe a biblioteca google.colab.drive
import google.colab.drive

# Monte o Google Drive
# Carregando os conjuntos de dados de treinamento e teste
google.colab.drive.mount('/content/drive')

path1 = '/content/drive/My Drive/df_Train_pronto.csv'
df_Train = pd.read_csv(path1)
display(df_Train)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ClasseDeConstrução,FrenteLote,AreaLote,QualidadeGeral,CondicaoGeral,AnoConstrucao,AnoRemodelacao,AreaAcabadaPorao1,AreaAcabadaPorao2,AreaNaoAcabadaPorao,...,Zoneamento_Utilidades_Funcionalidade_Cluster_10,Zoneamento_Utilidades_Funcionalidade_Cluster_11,Zoneamento_Utilidades_Funcionalidade_Cluster_12,Zoneamento_Utilidades_Funcionalidade_Cluster_13,Zoneamento_Utilidades_Funcionalidade_Cluster_14,Zoneamento_Utilidades_Funcionalidade_Cluster_15,Zoneamento_Utilidades_Funcionalidade_Cluster_16,Zoneamento_Utilidades_Funcionalidade_Cluster_17,Zoneamento_Utilidades_Funcionalidade_Cluster_18,Zoneamento_Utilidades_Funcionalidade_Cluster_19
0,60,65.0,8450,7,5,2003,2003,706.0,0.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20,80.0,9600,6,8,1976,1976,978.0,0.0,284.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,60,68.0,11250,7,5,2001,2002,486.0,0.0,434.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,70,60.0,9550,7,5,1915,1970,216.0,0.0,540.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,60,84.0,14260,8,5,2000,2000,655.0,0.0,490.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0.0,953.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1456,20,85.0,13175,6,6,1978,1988,790.0,163.0,589.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,70,66.0,9042,7,9,1941,2006,275.0,0.0,877.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1458,20,68.0,9717,5,6,1950,1996,49.0,1029.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Dividindo o conjunto de dados em conjunto de treinamento e conjunto de teste
X = df_Train.drop(['SalePrice'], axis=1)
y = df_Train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Padronização das features (variáveis independentes)
sc_X = StandardScaler(with_mean=False)
X_train_scaled = sc_X.fit_transform(X_train)
X_test_scaled = sc_X.transform(X_test)

# Normalização da variável dependente
sc_y = StandardScaler(with_mean=False)
y_train_scaled = sc_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()  # Utiliza o método ravel()
y_test_scaled = sc_y.transform(y_test.values.reshape(-1, 1)).ravel()  # Utiliza o método ravel()

# Hyperparâmetros a serem otimizados
n_estimators = [500, 900, 1400]
max_depth = [2, 3, 5]
learning_rate = [0.05, 0.1, 0.01]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 5]

# Define o grid de hiperparâmetros para pesquisa
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'learning_rate': learning_rate,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

# Inicializa o modelo Gradient Boosting Regressor
gb_model = GradientBoostingRegressor()

# Inicializa o GridSearchCV
grid_search = GridSearchCV(estimator=gb_model,
                           param_grid=hyperparameter_grid,
                           scoring='neg_mean_squared_error',
                           cv=3)
grid_result = grid_search.fit(X_train_scaled, y_train_scaled)

# Imprime os melhores hiperparâmetros encontrados
print("Best Parameters: ", grid_result.best_params_)

# Avalia o modelo nos dados de teste
y_pred_scaled = grid_search.predict(X_test_scaled)

# Avalie o desempenho do modelo no conjunto de teste
y_pred_scaled = grid_result.predict(X_test_scaled)

# Inverter a normalização e aplicar o logaritmo nos rótulos preditos
y_pred_original = sc_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
y_pred_log = np.log(y_pred_original)

# Aplicar o logaritmo nos rótulos reais
y_test_log = np.log(y_test)

# Avaliar o desempenho do modelo nos dados de teste
mse_test = mean_squared_error(y_test_log, y_pred_log)
r2_test = r2_score(y_test_log, y_pred_log)

print(f"Mean Squared Error on Test Set (logarithmic): {mse_test}")
print(f"R-squared on Test Set (logarithmic): {r2_test}")



Best Parameters:  {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500}
Mean Squared Error on Test Set (logarithmic): 0.0179030118723148
R-squared on Test Set (logarithmic): 0.8817501283402209


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Dividindo o conjunto de dados em conjunto de treinamento e conjunto de teste
X = df_Train.drop(['SalePrice'], axis=1)
y = df_Train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Padronização das features (variáveis independentes)
sc_X = StandardScaler(with_mean=False)
X_train_scaled = sc_X.fit_transform(X_train)
X_test_scaled = sc_X.transform(X_test)

# Normalização da variável dependente
sc_y = StandardScaler(with_mean=False)
y_train_scaled = sc_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()  # Utiliza o método ravel()
y_test_scaled = sc_y.transform(y_test.values.reshape(-1, 1)).ravel()  # Utiliza o método ravel()

# Melhores hiperparâmetros obtidos do GridSearchCV
best_params = {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 500}

# Inicializa o modelo Gradient Boosting Regressor com os melhores hiperparâmetros
best_gb_model = GradientBoostingRegressor(**best_params)

# Treina o modelo no conjunto de treinamento
best_gb_model.fit(X_train_scaled, y_train_scaled)

# Faz previsões nos dados de teste
y_pred_scaled = best_gb_model.predict(X_test_scaled)

# Inverter a normalização e aplicar o logaritmo nos rótulos preditos
y_pred_original = sc_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
y_pred_log = np.log(y_pred_original)

# Aplicar o logaritmo nos rótulos reais
y_test_log = np.log(y_test)

# Avaliar o desempenho do modelo nos dados de teste
mse_test = mean_squared_error(y_test_log, y_pred_log)
r2_test = r2_score(y_test_log, y_pred_log)

print(f"Mean Squared Error on Test Set (logarithmic): {mse_test}")
print(f"R-squared on Test Set (logarithmic): {r2_test}")


Mean Squared Error on Test Set (logarithmic): 0.015999755106687132
R-squared on Test Set (logarithmic): 0.894321190118888


In [ ]:
path1 = '/content/drive/My Drive/df_Train_pronto.csv'
df_Test = pd.read_csv(path1)

# Importar a coluna 'Id' do novo conjunto de dados de teste
path_test = '/content/drive/My Drive/test.csv'
df_Test_id = pd.read_csv(path_test, usecols=['Id'])

# Verificar e corrigir os dados da coluna 'Id'
df_Test_id['Id'] = pd.to_numeric(df_Test_id['Id'], errors='coerce')  # Converta para números e trate erros como NaN
df_Test_id['Id'] = df_Test_id['Id'].fillna(0).astype('int32')  # Preencha NaN com 0 e converta para int32

# Remover a coluna 'SalePrice' dos dados de teste
X_test = df_Test.drop(['SalePrice'], axis=1)

# Padronização das features (variáveis independentes)
X_test_scaled = sc_X.transform(X_test)  # Utilize o mesmo objeto sc_X usado no treinamento

# Fazer previsões nos dados de teste
y_pred_scaled = best_gb_model.predict(X_test_scaled)

# Inverter a normalização e aplicar o logaritmo nos rótulos preditos
y_pred_original = sc_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Adicionar a coluna 'Id' ao DataFrame existente df_Test
df_Test['Id'] = df_Test_id['Id']

# Criar DataFrame com Ids e previsões
submission_df = pd.DataFrame({'Id': df_Test['Id'], 'SalePrice': y_pred_original})

# Remover qualquer linha com valores nulos na coluna 'Id'
submission_df = submission_df.dropna(subset=['Id'])

# Forçar a coluna 'Id' para inteiros
submission_df['Id'] = submission_df['Id'].astype('int32')

# Salvar DataFrame como arquivo CSV
submission_df.to_csv('/content/drive/My Drive/submission5.csv', index=False)

In [ ]:
display(submission_df)

,Id,SalePrice
0,1461,203055.703940
1,1462,176744.380877
2,1463,206385.685241
3,1464,146238.076485
4,1465,276749.641147
...,...,...
1454,2915,193136.199349
1455,2916,171749.623070
1456,2917,209697.709893
1457,2918,262115.625984
